In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
pd.set_option('display.max_columns', 100)
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [167]:
movies_df = df[['Title','Genre','Director','Actors','Plot']]

The recommendations will be based on the 'Genre', 'Director' , 'Actors', 'Plot'

In [168]:
movies_df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [169]:
movies_df.shape

(250, 5)

In [170]:
#Take the Genre and create a list
movies_df['Genre'] = movies_df['Genre'].str.split(',')

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
movies_df['Genre']

0                 [Crime,  Drama]
1                 [Crime,  Drama]
2                 [Crime,  Drama]
3        [Action,  Crime,  Drama]
4                 [Crime,  Drama]
                  ...            
245           [Drama,  Film-Noir]
246                       [Drama]
247    [Comedy,  Drama,  Romance]
248           [Biography,  Drama]
249                       [Drama]
Name: Genre, Length: 250, dtype: object

In [172]:
#Take the directors and create a list
movies_df['Director'] = movies_df['Director'].str.split(',')

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [173]:
movies_df['Director']

0                     [Frank Darabont]
1               [Francis Ford Coppola]
2               [Francis Ford Coppola]
3                  [Christopher Nolan]
4                       [Sidney Lumet]
                    ...               
245                     [Billy Wilder]
246            [Destin Daniel Cretton]
247                     [Howard Hawks]
248                      [David Lynch]
249    [Danny Boyle,  Loveleen Tandan]
Name: Director, Length: 250, dtype: object

In [174]:
#Take the Actors and create the list
movies_df['Actors'] = movies_df['Actors'].apply(lambda x: x.split(',')[:3])

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [175]:
movies_df['Actors'] = movies_df['Actors']

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [176]:
#remove the space in between the first and last names in the director and the actor columns
movies_df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"[Crime, Drama]",[Frank Darabont],"[Tim Robbins, Morgan Freeman, Bob Gunton]",Two imprisoned men bond over a number of years...
1,The Godfather,"[Crime, Drama]",[Francis Ford Coppola],"[Marlon Brando, Al Pacino, James Caan]",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"[Crime, Drama]",[Francis Ford Coppola],"[Al Pacino, Robert Duvall, Diane Keaton]",The early life and career of Vito Corleone in ...
3,The Dark Knight,"[Action, Crime, Drama]",[Christopher Nolan],"[Christian Bale, Heath Ledger, Aaron Eckhart]",When the menace known as the Joker emerges fro...
4,12 Angry Men,"[Crime, Drama]",[Sidney Lumet],"[Martin Balsam, John Fiedler, Lee J. Cobb]",A jury holdout attempts to prevent a miscarria...


In [177]:
for index, rows in movies_df.iterrows():
    rows['Actors'] = [x.lower().replace(' ','') for x in rows['Actors']]
    rows['Director'] = [x.lower().replace(' ','') for x in rows['Director']]
    
   

In [178]:
movies_df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"[Crime, Drama]",[frankdarabont],"[timrobbins, morganfreeman, bobgunton]",Two imprisoned men bond over a number of years...
1,The Godfather,"[Crime, Drama]",[francisfordcoppola],"[marlonbrando, alpacino, jamescaan]",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"[Crime, Drama]",[francisfordcoppola],"[alpacino, robertduvall, dianekeaton]",The early life and career of Vito Corleone in ...
3,The Dark Knight,"[Action, Crime, Drama]",[christophernolan],"[christianbale, heathledger, aaroneckhart]",When the menace known as the Joker emerges fro...
4,12 Angry Men,"[Crime, Drama]",[sidneylumet],"[martinbalsam, johnfiedler, leej.cobb]",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...
245,The Lost Weekend,"[Drama, Film-Noir]",[billywilder],"[raymilland, janewyman, phillipterry]",The desperate life of a chronic alcoholic is f...
246,Short Term 12,[Drama],[destindanielcretton],"[brielarson, johngallagherjr., stephaniebeatriz]",A 20-something supervising staff member of a r...
247,His Girl Friday,"[Comedy, Drama, Romance]",[howardhawks],"[carygrant, rosalindrussell, ralphbellamy]",A newspaper editor uses every trick in the boo...
248,The Straight Story,"[Biography, Drama]",[davidlynch],"[sissyspacek, janegallowayheitz, josepha.carpe...",An old man makes a long journey by lawn-mover ...


In [179]:
#create a new column called bag of words which has the keywords corresponding to plot
movies_df['keywords'] = ""

for index, rows in movies_df.iterrows():
    plot = rows['Plot']
    
    #instantiate rake
    rake = Rake()
    
    rake.extract_keywords_from_text(plot)
    
    keywords = list(rake.get_word_degrees().keys())
    
    rows['keywords'] = keywords
    
movies_df.drop('Plot',axis=1,inplace=True)

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\91965\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [180]:
movies_df

,Title,Genre,Director,Actors,keywords
0,The Shawshank Redemption,"[Crime, Drama]",[frankdarabont],"[timrobbins, morganfreeman, bobgunton]","[finding, solace, common, decency, number, eve..."
1,The Godfather,"[Crime, Drama]",[francisfordcoppola],"[marlonbrando, alpacino, jamescaan]","[clandestine, empire, organized, crime, dynast..."
2,The Godfather: Part II,"[Crime, Drama]",[francisfordcoppola],"[alpacino, robertduvall, dianekeaton]","[michael, vito, corleone, grip, 1920s, new, yo..."
3,The Dark Knight,"[Action, Crime, Drama]",[christophernolan],"[christianbale, heathledger, aaroneckhart]","[menace, known, joker, emerges, fight, injusti..."
4,12 Angry Men,"[Crime, Drama]",[sidneylumet],"[martinbalsam, johnfiedler, leej.cobb]","[evidence, miscarriage, justice, jury, holdout..."
...,...,...,...,...,...
245,The Lost Weekend,"[Drama, Film-Noir]",[billywilder],"[raymilland, janewyman, phillipterry]","[followed, chronic, alcoholic, day, drinking, ..."
246,Short Term 12,[Drama],[destindanielcretton],"[brielarson, johngallagherjr., stephaniebeatriz]","[world, alongside, longtime, boyfriend, co, tr..."
247,His Girl Friday,"[Comedy, Drama, Romance]",[howardhawks],"[carygrant, rosalindrussell, ralphbellamy]","[remarrying, newspaper, editor, uses, every, t..."
248,The Straight Story,"[Biography, Drama]",[davidlynch],"[sissyspacek, janegallowayheitz, josepha.carpe...","[relationship, ill, brother, old, man, makes, ..."


In [181]:
#set the title as index and combine rest of the details which will be used for the cosine similarity comparison
movies_df.set_index('Title', inplace=True)

In [182]:
movies_df['bagofwords'] = ""
cols = movies_df.columns

#combine all the columns into single columns called bag of words
for index, rows in movies_df.iterrows():
    rows['bagofwords'] = rows['Genre'] + rows['Director'] + rows['Actors'] + rows['keywords']
    
        

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [183]:
for index, rows in movies_df.iterrows():
    word = ''
    for i in rows['bagofwords']:
        word = word + ' ' + i
    rows['bagofwords'] = word

In [184]:
movies_df = pd.DataFrame(movies_df['bagofwords'])

In [185]:
movies_df.head()

,bagofwords
Title,
The Shawshank Redemption,Crime Drama frankdarabont timrobbins morganf...
The Godfather,Crime Drama francisfordcoppola marlonbrando ...
The Godfather: Part II,Crime Drama francisfordcoppola alpacino robe...
The Dark Knight,Action Crime Drama christophernolan christi...
12 Angry Men,Crime Drama sidneylumet martinbalsam johnfie...


In [187]:
#now use the count vectorizer to create a document matrix
countvectorizer = CountVectorizer()
count = countvectorizer.fit_transform(movies_df['bagofwords'])

In [189]:
count.todense()

(250, 2961)

In [192]:
final_df = pd.DataFrame(cosine_similarity(count, count), columns = movies_df.index, index = movies_df.index)

In [193]:
final_df.head()

Title,The Shawshank Redemption,The Godfather,The Godfather: Part II,The Dark Knight,12 Angry Men,Schindler's List,The Lord of the Rings: The Return of the King,Pulp Fiction,Fight Club,The Lord of the Rings: The Fellowship of the Ring,Forrest Gump,Star Wars: Episode V - The Empire Strikes Back,Inception,The Lord of the Rings: The Two Towers,One Flew Over the Cuckoo's Nest,Goodfellas,The Matrix,Star Wars: Episode IV - A New Hope,Se7en,It's a Wonderful Life,The Silence of the Lambs,The Usual Suspects,Léon: The Professional,Saving Private Ryan,City Lights,Interstellar,American History X,Modern Times,Casablanca,The Green Mile,Psycho,Raiders of the Lost Ark,The Pianist,Rear Window,The Departed,Whiplash,Terminator 2: Judgment Day,Back to the Future,Gladiator,The Lion King,The Prestige,Apocalypse Now,Memento,The Great Dictator,Sunset Boulevard,Alien,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,Paths of Glory,Django Unchained,The Shining,...,Dog Day Afternoon,Jaws,Zootopia,Guardians of the Galaxy Vol. 2,Pirates of the Caribbean: The Curse of the Black Pearl,Before Sunset,The Imitation Game,Young Frankenstein,Stalag 17,Dogville,Dead Poets Society,High Noon,Papillon,A Streetcar Named Desire,Arsenic and Old Lace,Sin City,The Hustler,A Night at the Opera,The Killing,The Avengers,Notorious,Harvey,The Martian,The Exorcist,Rio Bravo,The Philadelphia Story,Rope,The Big Sleep,Pink Floyd: The Wall,The King's Speech,A Christmas Story,The Graduate,JFK,Sling Blade,Blood Diamond,Magnolia,Rain Man,The Revenant,The Nightmare Before Christmas,The Manchurian Candidate,Deadpool,The Wild Bunch,Aladdin,Big Fish,Patton,The Lost Weekend,Short Term 12,His Girl Friday,The Straight Story,Slumdog Millionaire
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,1.000000,0.157895,0.137649,0.083771,0.111283,0.046829,0.095673,0.244558,0.044992,0.046829,0.051299,0.000000,0.000000,0.045883,0.052632,0.170406,0.00000,0.039936,0.210526,0.051299,0.097823,0.091766,0.089984,0.050063,0.046829,0.055641,0.108148,0.052632,0.054074,0.143509,0.0,0.000000,0.054074,0.000000,0.108148,0.045883,0.000000,0.040555,0.051299,0.046829,0.105263,0.046829,0.000000,0.050063,0.050063,0.000000,0.0,0.054074,0.052632,0.046829,...,0.108148,0.041204,0.000000,0.000000,0.000000,0.111283,0.051299,0.000000,0.091766,0.086711,0.052632,0.048912,0.146735,0.052632,0.102598,0.047836,0.055641,0.047836,0.118470,0.000000,0.052632,0.043355,0.047836,0.054074,0.045883,0.0,0.220755,0.048912,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.000000,0.0,0.057354,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
The Godfather,0.157895,1.000000,0.367065,0.125656,0.166924,0.046829,0.047836,0.146735,0.044992,0.046829,0.051299,0.041885,0.000000,0.045883,0.052632,0.213007,0.00000,0.039936,0.157895,0.051299,0.146735,0.137649,0.134976,0.050063,0.046829,0.055641,0.162221,0.052632,0.054074,0.143509,0.0,0.000000,0.054074,0.000000,0.162221,0.045883,0.047836,0.000000,0.102598,0.046829,0.052632,0.140488,0.000000,0.100125,0.050063,0.000000,0.0,0.054074,0.052632,0.093659,...,0.162221,0.041204,0.000000,0.000000,0.000000,0.055641,0.051299,0.000000,0.045883,0.130066,0.052632,0.048912,0.146735,0.105263,0.153897,0.095673,0.055641,0.000000,0.177705,0.000000,0.052632,0.043355,0.047836,0.000000,0.045883,0.0,0.220755,0.097823,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.051299,0.0,0.114708,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
The Godfather: Part II,0.137649,0.367065,1.000000,0.109545,0.145521,0.040825,0.041703,0.127920,0.078446,0.040825,0.044721,0.000000,0.000000,0.080000,0.045883,0.259973,0.00000,0.000000,0.137649,0.134164,0.127920,0.120000,0.156893,0.043644,0.040825,0.048507,0.141421,0.091766,0.047140,0.125109,0.0,0.000000,0.047140,0.000000,0.141421,0.040000,0.041703,0.000000,0.1

In [198]:
#print the first 10 recommendations for the shawshank redemption
final_df.loc['Fargo'].sort_values(ascending=False)[0:11]

Title
Fargo                     1.000000
No Country for Old Men    0.230940
The Departed              0.226805
Rope                      0.222222
The Godfather             0.220755
Reservoir Dogs            0.215166
The Godfather: Part II    0.192450
On the Waterfront         0.181444
Goodfellas                0.178685
Arsenic and Old Lace      0.172133
The Big Lebowski          0.172133
Name: Fargo, dtype: float64